<a href="https://colab.research.google.com/github/msbeigi/sale-analysis/blob/main/backery_sale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [143]:
import pandas as pd
import numpy as np
from IPython.display import display,HTML
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

## Initiation and Data Understanding

### Load Data

In [131]:
weather=pd.read_csv("/content/drive/MyDrive/ML/Hand-on/Backery/weather_2021.01.01-2022.10.31.csv")
backery=pd.read_csv("/content/drive/MyDrive/ML/Hand-on/Backery/bakery_sales_2021-2022.csv")
title='<h3>Sample content of Datsets:</h3>'
display(HTML(title))
title='<h3>bakery_sales_2021-2022.csv</h3>'
display(HTML(title))
display(backery.head(7))
title='<h3>weather_2021.01.01-2022.10.31.csv</h3>'
display(HTML(title))
display(weather.head(7))

,Unnamed: 0,date,time,ticket_number,Article,Quantity,UNIT_PRICE
0,0,2021-01-02,08:38,150040.0,BAGUETTE,1.0,"0,90 €"
1,1,2021-01-02,08:38,150040.0,PAIN AU CHOCOLAT,3.0,"1,20 €"
2,4,2021-01-02,09:14,150041.0,PAIN AU CHOCOLAT,2.0,"1,20 €"
3,5,2021-01-02,09:14,150041.0,PAIN,1.0,"1,15 €"
4,8,2021-01-02,09:25,150042.0,TRADITIONAL BAGUETTE,5.0,"1,20 €"
5,11,2021-01-02,09:25,150043.0,BAGUETTE,2.0,"0,90 €"
6,12,2021-01-02,09:25,150043.0,CROISSANT,3.0,"1,10 €"


,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2021-01-01,2.7,1.5,5.7,15.2,NaN,335.0,15.5,31.0,1008.6,NaN
1,2021-01-02,1.3,0.5,2.7,0.8,NaN,336.0,24.7,44.0,1010.1,NaN
2,2021-01-03,0.7,-0.3,1.6,0.0,NaN,327.0,17.1,38.9,1012.1,NaN
3,2021-01-04,0.0,-1.0,1.1,0.0,NaN,329.0,10.6,30.0,1011.4,NaN
4,2021-01-05,0.8,-0.9,2.1,0.0,NaN,338.0,6.2,30.0,1012.1,NaN
5,2021-01-06,0.8,0.2,1.4,0.0,NaN,324.0,8.0,21.0,1015.4,NaN
6,2021-01-07,0.2,-1.6,2.5,0.0,NaN,343.0,6.2,24.0,1017.1,NaN


### Summary Missing Values and Feature Types

In [61]:
meta_backery=['row index' 	,'date of sale ','time of sale ', 'receipt number ','product name ', 'quantity of the product','unit price of the product']
meta_weahter=['The date of the measurement', 'The average air temperature in °C', 'The minimum air temperature in °C','The maximum air temperature in °C'
              ,'The daily precipitation total in mm', 'The snow depth in mm', 'The average wind direction in degrees (°)', 'The average wind speed in km/h'
              ,'The peak wind gust in km/h', 'The average sea-level air pressure in hPa', 'The daily sunshine total in minutes (m)']


missing_values=backery.isna().sum()
missing_values_weather=weather.isna().sum()

feature_typ=backery.dtypes
feature_typ_weather=weather.dtypes

# out put for backery dataset
_=zip(missing_values.index,meta_backery,missing_values.values,feature_typ.values)
df_backer_desc = pd.DataFrame(_, columns=['column','feature_escription', 'number_of_missing_values', 'feature_type']).style.set_properties(**{'text-align': 'left'})
title = "<h3>Backery Dataset Summary with Missing Values and Data Types</h3>"
display(HTML(title))
display(df_backer_desc)
# output for weather dataset
_=zip(missing_values_weather.index,meta_weahter,missing_values_weather.values,feature_typ_weather.values)
df_weather_desc = pd.DataFrame(_, columns=['column','feature_description', 'number_of_missing_values', 'feature_type']).style.set_properties(**{'text-align': 'left'})
title = "<h3>Weather Dataset Summary with Missing Values and Data Types</h3>"
display(HTML(title))
display(df_weather_desc)


,column,feature_escription,number_of_missing_values,feature_type
0,Unnamed: 0,row index,0,int64
1,date,date of sale,0,object
2,time,time of sale,0,object
3,ticket_number,receipt number,0,float64
4,Article,product name,0,object
5,Quantity,quantity of the product,0,float64
6,UNIT_PRICE,unit price of the product,0,object


,column,feature_description,number_of_missing_values,feature_type
0,date,The date of the measurement,0,object
1,tavg,The average air temperature in °C,0,float64
2,tmin,The minimum air temperature in °C,0,float64
3,tmax,The maximum air temperature in °C,0,float64
4,prcp,The daily precipitation total in mm,0,float64
5,snow,The snow depth in mm,654,float64
6,wdir,The average wind direction in degrees (°),2,float64
7,wspd,The average wind speed in km/h,2,float64
8,wpgt,The peak wind gust in km/h,14,float64
9,pres,The average sea-level air pressure in hPa,2,float64


### Statistical Review

In [50]:
title='<h3>Statistical review</h3>'
display(HTML(title))
title='<h4>backery:</h4>'
display(HTML(title))
display(backery.describe())
title='<h4>weather:</h4>'
display(HTML(title))
display(weather.describe())


,Unnamed: 0,ticket_number,Quantity
count,234005.000000,234005.000000,234005.000000
mean,255205.035854,219201.258738,1.538377
std,147501.625992,40053.223896,1.289603
min,0.000000,150040.000000,-200.000000
25%,127979.000000,184754.000000,1.000000
50%,254573.000000,218807.000000,1.000000
75%,382911.000000,253927.000000,2.000000
max,511395.000000,288913.000000,200.000000


,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
count,669.000000,669.000000,669.000000,669.000000,15.000000,667.000000,667.000000,655.000000,667.000000,0.0
mean,13.900747,9.552167,18.830344,2.272945,14.733333,225.757121,12.674813,39.014198,1018.422789,NaN
std,7.710498,6.918623,8.864005,7.402603,9.353889,108.937433,6.022562,14.485416,7.056279,NaN
min,-3.300000,-5.400000,-1.500000,0.000000,10.000000,0.000000,3.100000,11.000000,996.400000,NaN
25%,7.800000,3.800000,12.400000,0.000000,10.000000,155.000000,8.000000,28.000000,1014.200000,NaN
50%,14.200000,9.800000,19.500000,0.000000,10.000000,192.000000,11.300000,37.000000,1018.400000,NaN
75%,20.000000,15.600000,25.500000,0.800000,15.000000,338.500000,16.000000,46.300000,1022.300000,NaN
max,31.800000,24.800000,38.500000,103.400000,41.000000,359.000000,41.000000,93.000000,1039.300000,NaN


## Data preparation

### Missing Values

In [121]:
def drop_nonvalue(df :pd.DataFrame)->pd.DataFrame:
  """
  Args:
    df: weather dataset as dataframe input
  Returns:
    returns cleared dataframe of missing values
  """
  df.drop(['tsun','snow'], axis=1,inplace=True)

  return df

def impute_missing_values(df:pd.DataFrame)->pd.DataFrame:
    """
    Args:
      df: weather dataset as dataframe input
    Returns:
      return the modified DataFrame
    """
    columns_tobe_fill = ['wdir','wspd','wpgt','pres']
    imputer = IterativeImputer(estimator=RandomForestRegressor(), max_iter=10, random_state=42)
    df[columns_tobe_fill] = imputer.fit_transform(df[columns_tobe_fill])


    return df

In [132]:
df_weather=weather.copy()
df_weather=drop_nonvalue(df_weather)
df_weather=impute_missing_values(df_weather)
print('Number of missing values:\n----------------')
print(df_weather.isna().sum())


Number of missing values:
----------------
date    0
tavg    0
tmin    0
tmax    0
prcp    0
wdir    0
wspd    0
wpgt    0
pres    0
dtype: int64


### Feature Engineering

In [152]:
backery[backery['Quantity']<0]

,Unnamed: 0,date,time,ticket_number,Article,Quantity,UNIT_PRICE
448,898,2021-01-03,10:41,150265.0,BAGUETTE,-1.0,"0,90 €"
1162,2558,2021-01-05,19:00,150738.0,SPECIAL BREAD,-1.0,"2,40 €"
1163,2559,2021-01-05,19:00,150738.0,DIVERS PATISSERIE,-1.0,"6,00 €"
1164,2560,2021-01-05,19:00,150738.0,DIVERS BOULANGERIE,-1.0,"2,00 €"
1467,3269,2021-01-08,11:58,150941.0,COUPE,-1.0,"0,15 €"
...,...,...,...,...,...,...,...
233054,509243,2022-09-27,18:03,288312.0,COUPE,-2.0,"0,15 €"
233055,509244,2022-09-27,18:03,288312.0,TRADITIONAL BAGUETTE,-1.0,"1,30 €"
233056,509245,2022-09-27,18:03,288312.0,CEREAL BAGUETTE,-3.0,"1,35 €"
233553,510396,2022-09-29,11:55,288639.0,BOISSON 33CL,-1.0,"1,50 €"


In [151]:
backery[backery['UNIT_PRICE']=='0,00 €']

,Unnamed: 0,date,time,ticket_number,Article,Quantity,UNIT_PRICE
78,156,2021-01-02,10:51,150079.0,DIVERS PATISSERIE,1.0,"0,00 €"
10373,23009,2021-02-19,10:13,156355.0,DIVERS BOULANGERIE,1.0,"0,00 €"
15360,33726,2021-03-04,12:32,159219.0,.,2.0,"0,00 €"
15946,34951,2021-03-05,18:18,159538.0,GD FAR BRETON,1.0,"0,00 €"
19904,43541,2021-03-18,12:59,161853.0,.,1.0,"0,00 €"
24959,54650,2021-04-04,09:53,164878.0,.,1.0,"0,00 €"
33569,73667,2021-04-27,16:48,170079.0,.,1.0,"0,00 €"
34671,76164,2021-05-01,10:17,170776.0,DIVERS PATISSERIE,1.0,"0,00 €"
39840,87369,2021-05-13,11:15,173793.0,DIVERS CONFISERIE,1.0,"0,00 €"
61792,135091,2021-07-10,13:25,186662.0,.,2.0,"0,00 €"


In [149]:
backery[backery['Article']=='.']

,Unnamed: 0,date,time,ticket_number,Article,Quantity,UNIT_PRICE
15360,33726,2021-03-04,12:32,159219.0,.,2.0,"0,00 €"
19904,43541,2021-03-18,12:59,161853.0,.,1.0,"0,00 €"
24959,54650,2021-04-04,09:53,164878.0,.,1.0,"0,00 €"
33569,73667,2021-04-27,16:48,170079.0,.,1.0,"0,00 €"
61792,135091,2021-07-10,13:25,186662.0,.,2.0,"0,00 €"


In [148]:
backery['Article'].unique()

array(['BAGUETTE', 'PAIN AU CHOCOLAT', 'PAIN', 'TRADITIONAL BAGUETTE',
       'CROISSANT', 'BANETTE', 'BANETTINE', 'SPECIAL BREAD', 'COUPE',
       'SAND JB EMMENTAL', 'KOUIGN AMANN', 'BOULE 200G', 'BOULE 400G',
       'GAL FRANGIPANE 6P', 'CAMPAGNE', 'MOISSON', 'CAFE OU EAU',
       'BRIOCHE', 'CEREAL BAGUETTE', 'SEIGLE', 'COMPLET',
       'DIVERS PATISSERIE', 'GAL FRANGIPANE 4P', 'COOKIE', 'FICELLE',
       'PAIN AUX RAISINS', 'GAL POMME 6P', 'GAL POMME 4P', 'FINANCIER X5',
       'VIK BREAD', 'DIVERS VIENNOISERIE', 'GACHE', 'SANDWICH COMPLET',
       'PAIN BANETTE', 'GRAND FAR BRETON', 'QUIM BREAD',
       'SPECIAL BREAD KG', 'GD KOUIGN AMANN', 'BOULE POLKA',
       'DEMI BAGUETTE', 'CHAUSSON AUX POMMES', 'BAGUETTE GRAINE',
       'DIVERS CONFISERIE', 'SUCETTE', 'DIVERS BOULANGERIE',
       'BOISSON 33CL', 'PATES', 'FORMULE SANDWICH', 'DIVERS SANDWICHS',
       'CROISSANT AMANDES', 'PAIN CHOCO AMANDES', 'SACHET VIENNOISERIE',
       'NANTAIS', 'CHOCOLAT', 'PAIN S/SEL', 'FONDANT CHOC

In [146]:
encoder = OneHotEncoder(sparse=False)
article_encoded = encoder.fit_transform(backery[['Article']])
encode_columns=encoder.get_feature_names_out()
encoded_df=pd.DataFrame(article_encoded,columns=encode_columns)

,Article_.,Article_12 MACARON,Article_ARMORICAIN,Article_ARTICLE 295,Article_BAGUETTE,...,Article_TROPEZIENNE,Article_TROPEZIENNE FRAMBOISE,Article_TULIPE,Article_VIENNOISE,Article_VIK BREAD
0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
234000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
234001,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
234002,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
234003,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0


In [140]:


def added_feature_bakery(df: pd.DataFrame)-> pd.DataFrame:
  """
  Args:
   df: bakery dataset as dataframe input
  Returns:
    return new dataset with added feature
  """
  #categorical features

  # numerical features
  df['unit_price_value']= df['UNIT_PRICE'].apply(lambda x: extract_price(x))
  df["total_sale_item"]= df['Quantity']*df['unit_price_value']
  df=df.rename(columns={'Unnamed: 0':'index'})
  df['hour_day']=df['time'].apply(lambda x: pd.to_datetime(x, format='%H:%M').hour)
  df['day_of_week'] = df['date'].apply(lambda x: pd.to_datetime(x,format='%Y-%m-%d').dayofweek)
  df['day_name']= df['date'].apply(lambda x: pd.to_datetime(x,format='%Y-%m-%d').day_name())
  df['is_weekend'] = df['day_name'].isin(['Saturday', 'Sunday']).astype(int)

  return df

def added_feature_weather(df: pd.DataFrame)-> pd.DataFrame:
  """
  Args:
   df: weather dataset as dataframe input
  Returns:
    return new dataset with added feature
  """
  df['is_rainy']=df['prcp']>0
  df['temp_range'] = df['tmax'] - df['tmin']
  df['is_cold_day'] = (df['tavg'] < 10).astype(int)
  df['is_hot_day'] = (df['tavg'] > 25).astype(int)
  df['is_heavy_wind'] = (df['wspd'] > 20).astype(int)
  df['is_high_pressure'] = (df['pres'] > 1015).astype(int)

  return df

def extract_price(price_tag: object)->float:
  """
  Args:
    price_tag: string input for daily sale with currency sign

  Returns:
    returns price value as float
  """
  price_string=str(price_tag)
  price_value=float(price_string[:-2].replace(',','.'))
  return price_value

In [141]:
df_bekery=backery.copy()
df_bekery=added_feature_bakery(df_bekery)
df_weather=added_feature_weather(df_weather)

In [142]:
display(df_weather)
display(df_bekery)

,date,tavg,tmin,tmax,prcp,wdir,wspd,wpgt,pres,is_rainy,temp_range,is_cold_day,is_hot_day,is_heavy_wind,is_high_pressure
0,2021-01-01,2.7,1.5,5.7,15.2,335.0,15.5,31.0,1008.6,True,4.2,1,0,0,0
1,2021-01-02,1.3,0.5,2.7,0.8,336.0,24.7,44.0,1010.1,True,2.2,1,0,1,0
2,2021-01-03,0.7,-0.3,1.6,0.0,327.0,17.1,38.9,1012.1,False,1.9,1,0,0,0
3,2021-01-04,0.0,-1.0,1.1,0.0,329.0,10.6,30.0,1011.4,False,2.1,1,0,0,0
4,2021-01-05,0.8,-0.9,2.1,0.0,338.0,6.2,30.0,1012.1,False,3.0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,2022-10-27,19.8,15.7,25.0,0.0,145.0,17.4,39.0,1023.8,False,9.3,0,0,0,1
665,2022-10-28,20.9,17.4,25.5,0.0,136.0,16.3,39.0,1024.5,False,8.1,0,0,0,1
666,2022-10-29,19.6,14.9,25.5,0.0,141.0,14.3,38.9,1022.7,False,10.6,0,0,0,1
667,2022-10-30,18.8,15.4,24.4,0.0,136.0,17.0,33.0,1020.4,False,9.0,0,0,0,1


,index,date,time,ticket_number,Article,Quantity,UNIT_PRICE,unit_price_value,total_sale_item,hour_day,day_of_week,day_name,is_weekend
0,0,2021-01-02,08:38,150040.0,BAGUETTE,1.0,"0,90 €",0.90,0.90,8,5,Saturday,1
1,1,2021-01-02,08:38,150040.0,PAIN AU CHOCOLAT,3.0,"1,20 €",1.20,3.60,8,5,Saturday,1
2,4,2021-01-02,09:14,150041.0,PAIN AU CHOCOLAT,2.0,"1,20 €",1.20,2.40,9,5,Saturday,1
3,5,2021-01-02,09:14,150041.0,PAIN,1.0,"1,15 €",1.15,1.15,9,5,Saturday,1
4,8,2021-01-02,09:25,150042.0,TRADITIONAL BAGUETTE,5.0,"1,20 €",1.20,6.00,9,5,Saturday,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
234000,511387,2022-09-30,18:52,288911.0,COUPE,1.0,"0,15 €",0.15,0.15,18,4,Friday,0
234001,511388,2022-09-30,18:52,288911.0,BOULE 200G,1.0,"1,20 €",1.20,1.20,18,4,Friday,0
234002,511389,2022-09-30,18:52,288911.0,COUPE,2.0,"0,15 €",0.15,0.30,18,4,Friday,0
234003,511392,2022-09-30,18:55,288912.0,TRADITIONAL BAGUETTE,1.0,"1,30 €",1.30,1.30,18,4,Friday,0


In [ ]:
merged_df['temp_range'] = merged_df['tmax'] - merged_df['tmin']
merged_df['is_cold_day'] = (merged_df['tavg'] < 10).astype(int)
merged_df['is_hot_day'] = (merged_df['tavg'] > 25).astype(int)
merged_df['is_rainy_day'] = (merged_df['prcp'] > 0).astype(int)
merged_df['is_snowy_day'] = (merged_df['snow'] > 0).astype(int)
merged_df['high_wind_speed'] = (merged_df['wspd'] > 20).astype(int)
merged_df['wind_gust_presence'] = (merged_df['wpgt'] > 0).astype(int)
merged_df['high_sunshine_hours'] = (merged_df['tsun'] > 300).astype(int)
merged_df['high_pressure'] = (merged_df['pres'] > 1015).astype(int)


In [58]:

daily_count=backery.groupby(["date"])["Article"].nunique().reset_index(name='unique_items_sold')#.count()

display(daily_count)


print('Mean items sold per day:\n')
print(np.mean( daily_count['unique_items_sold'].values))

,date,unique_items_sold
0,2021-01-02,40
1,2021-01-03,38
2,2021-01-04,34
3,2021-01-05,39
4,2021-01-07,41
...,...,...
595,2022-09-26,40
596,2022-09-27,45
597,2022-09-28,46
598,2022-09-29,37


Mean items sold per day:

48.67


In [65]:
merged_data = pd.merge(daily_count, weather, on='date', how='left')
merged_data

,date,unique_items_sold,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,is_rainy,is_hot_day,is_freezing_day,is_cloudy,is_heavy_wind
0,2021-01-02,40,1.3,0.5,2.7,0.8,NaN,336.0,24.7,44.0,1010.1,NaN,True,False,False,False,True
1,2021-01-03,38,0.7,-0.3,1.6,0.0,NaN,327.0,17.1,38.9,1012.1,NaN,False,False,False,False,True
2,2021-01-04,34,0.0,-1.0,1.1,0.0,NaN,329.0,10.6,30.0,1011.4,NaN,False,False,False,False,False
3,2021-01-05,39,0.8,-0.9,2.1,0.0,NaN,338.0,6.2,30.0,1012.1,NaN,False,False,False,False,False
4,2021-01-07,41,0.2,-1.6,2.5,0.0,NaN,343.0,6.2,24.0,1017.1,NaN,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,2022-09-26,40,14.8,11.8,18.3,2.8,NaN,297.0,7.8,22.2,1011.3,NaN,True,False,False,False,False
596,2022-09-27,45,13.4,10.7,16.3,3.6,NaN,240.0,10.5,33.0,1008.6,NaN,True,False,False,True,True
597,2022-09-28,46,14.2,11.2,19.2,4.3,NaN,190.0,15.9,48.0,1003.1,NaN,True,False,False,True,True
598,2022-09-29,37,12.2,9.3,15.7,24.4,NaN,321.0,6.9,18.5,1003.2,NaN,True,False,False,True,False


## EDA

In [100]:
import plotly.graph_objects as go
conditions = ['is_rainy', 'is_hot_day', 'is_freezing_day', 'is_cloudy', 'is_heavy_wind']

colors = {
    # 'is_rainy': 'lightblue',
    # 'is_hot_day': 'red',
    # 'is_freezing_day': 'cyan',
    'is_cloudy': 'gray',
    'is_heavy_wind': 'lightseagreen'
}
markers = {'is_rainy': 'circle',
           'is_freezing_day':  'diamond',
           'is_hot_day':'cross'
}
marker_colors = {'is_rainy': 'darkblue',
           'is_freezing_day':  'cyan',
           'is_hot_day':'red'
}

fig = go.Figure()
# fig.add_trace(go.Scatter(
#     x=merged_data['date'],
#     y=merged_data['unique_items_sold'],
#     mode='markers+lines',
#     name='Unique Items Sold'
# ))

for condition in markers.keys():
    if condition in merged_data.columns:
        condition_data = merged_data[merged_data[condition]]

        fig.add_trace(go.Scatter(
            x=merged_data['date'],
            y=merged_data['unique_items_sold'],
            mode='markers',
            marker=dict(
                symbol=markers[condition],
                size=10,
                color=marker_colors[condition],
                opacity=0.8,
                line=dict(width=0)
            ),
            name=condition.replace('_', ' ').title(),
            showlegend=True,
            hoverinfo='name'
        ))


shapes = []
if any(merged_data['is_cloudy']):
    shapes.append({
        'type': 'rect',
        'xref': 'x',
        # 'yref': 'paper',
        'x0': merged_data.loc[merged_data['is_cloudy'], 'date'].min(),
        'x1': merged_data.loc[merged_data['is_cloudy'], 'date'].max(),
        'y0': 0,
        'y1': 1,
        'fillcolor': colors['is_cloudy'],
        'opacity': 0.6,
        # 'layer': 'above',
        # 'line': {'width': 1},
    })


if any(merged_data['is_heavy_wind']):
    shapes.append({
        'type': 'rect',
        # 'xref': 'x',
        'yref': 'paper',
        'x0': merged_data.loc[merged_data['is_heavy_wind'], 'date'].min(),
        'x1': merged_data.loc[merged_data['is_heavy_wind'], 'date'].max(),
        'y0': merged_data['unique_items_sold'].min(),
        'y1': merged_data['unique_items_sold'].max(),
        'fillcolor': colors['is_heavy_wind'],
        'opacity':0.6,
        # 'layer': 'above',
        # 'line': {'width': 1},
    })

fig.update_layout( title='Daily Unique Items Sold', xaxis_title='Date',  yaxis_title='Unique Items Sold',  shapes=shapes )
fig.show()




In [43]:

daily_sale=backery.groupby(["date","Article"]).sum("Quantity")#.size()
daily_sale

Id  ticket_number  Quantity  \
date       Article                                                   
2021-01-02 BAGUETTE                 10519      4353814.0      46.0   
           BANETTE                   8968      3753253.0      40.0   
           BANETTINE                 1298       750529.0       6.0   
           BOULE 200G                3428       901109.0       6.0   
           BOULE 400G                4548      1351512.0      11.0   
...                                   ...            ...       ...   
2022-09-30 SPECIAL BREAD KG        511210       288863.0       1.0   
           SUCETTE                 511268       288877.0       1.0   
           TARTELETTE             1021867       577575.0       3.0   
           TRADITIONAL BAGUETTE  43441397     24550190.0     141.0   
           VIK BREAD              3577175      2021686.0      12.0   

                                 UNIT_PRICE_value  total_sale_item  
date       Article                                                  
2021-01-02 BAGUETTE                         26.10             41.4  
           BANETTE                          26.25             42.0  
           BANETTINE                         3.00              3.6  
           BOULE 200G                        6.60              6.6  
           BOULE 400G                       13.50             16.5  
...                                           ...              ...  
2022-09-30 SPECIAL BREAD KG                  5.20              5.2  
           SUCETTE                           0.30              0.3  
           TARTELETTE                        4.40              6.6  
           TRADITIONAL BAGUETTE            110.50            183.3  
           VIK BREAD                        18.90             32.4  

[29202 rows x 5 columns]